In [ ]:
import libspn as spn
import tensorflow as tf

# Generate Dense SPN with Random Weights

In [ ]:
iv=spn.IVs(num_vars=3, num_vals=2)
gen=spn.DenseSPNGeneratorMultiNodes(num_decomps=1, num_subsets=3, num_mixtures=2)
root=gen.generate(iv, root_name="root")
spn.generate_weights(root, init_value=spn.ValueType.RANDOM_UNIFORM())

In [ ]:
spn.display_spn_graph(root, skip_params=False)

# Save With Uninitialized Weights

In [ ]:
spn.JSONSaver('saved_models/test_uninit.spn', pretty=True).save(root)

# Initialize and Save With Initialized Weights

In [ ]:
init=spn.initialize_weights(root)
value = root.get_value()
with tf.Session() as sess:
    sess.run(init)
    first_value = sess.run(value, feed_dict={iv: [[-1, -1, -1],
                                                  [0,0,0],
                                                  [1,1,1]]})
    print(first_value)
    spn.JSONSaver('saved_models/test_init.spn').save(root)

# Load and Run the SPN With Uninitialzed Weights

In [ ]:
loader=spn.JSONLoader('saved_models/test_uninit.spn')
root=loader.load()
print(root)

In [ ]:
spn.display_spn_graph(root, skip_params=False)

In [ ]:
init=spn.initialize_weights(root)
value = root.get_value()
iv = loader.find_node("IVs")
with tf.Session() as sess:
    sess.run(init)
    second_value = sess.run(value, feed_dict={iv: [[-1, -1, -1],
                                                  [0,0,0],
                                                  [1,1,1]]})
    print(first_value)
    print(second_value)

We see that the weights of the loaded SPN were initialized after loading to random uniform distribution, since they were not initialized before saving! Therefore, second_value differs from first_value.

# Load and Run the SPN With Initialzed Weights

In [ ]:
loader=spn.JSONLoader('saved_models/test_init.spn')
root=loader.load()
print(root)

In [ ]:
spn.display_spn_graph(root, skip_params=False)

In [ ]:
init=spn.initialize_weights(root)
value = root.get_value()
iv = loader.find_node("IVs")
with tf.Session() as sess:
    sess.run(init)
    third_value = sess.run(value, feed_dict={iv: [[-1, -1, -1],
                                                  [0,0,0],
                                                  [1,1,1]]})
    print(first_value)
    print(third_value)

We see that now the variables are initialized to the values to which they were initialized before. Therefore, we get the same result.